# Tensorflow Object Detection API and AWS Sagemaker

In this notebook, you will train and evaluate different models using the [Tensorflow Object Detection API](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/) and [AWS Sagemaker](https://aws.amazon.com/sagemaker/). 

If you ever feel stuck, you can refer to this [tutorial](https://aws.amazon.com/blogs/machine-learning/training-and-deploying-models-using-tensorflow-2-with-the-object-detection-api-on-amazon-sagemaker/).

## Dataset

We are using the [Waymo Open Dataset](https://waymo.com/open/) for this project. The dataset has already been exported using the tfrecords format. The files have been created following the format described [here](https://tensorflow-object-detection-api-tutorial.readthedocs.io/en/latest/training.html#create-tensorflow-records). You can find data stored on [AWS S3](https://aws.amazon.com/s3/), AWS Object Storage. The images are saved with a resolution of 640x640.

In [1]:
%%capture
%pip install tensorflow_io sagemaker -U

In [2]:
import os
import sagemaker
from sagemaker.estimator import Estimator
from framework import CustomFramework

/home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/pydantic/_internal/_fields.py:172: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[12/23/24 17:25:58] INFO     Found credentials from IAM Role:                                   ]8;id=632013;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=53974;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


Save the IAM role in a variable called `role`. This would be useful when training the model.

In [3]:
role = sagemaker.get_execution_role()
print(role)

[12/23/24 17:26:03] INFO     Found credentials from IAM Role:                                   ]8;id=724113;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=936151;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

arn:aws:iam::796973520885:role/service-role/AmazonSageMaker-ExecutionRole-20241219T112240


In [4]:
# The train and val paths below are public S3 buckets created by Udacity for this project
inputs = {'train': 's3://cd2688-object-detection-tf2/train/', 
          'val': 's3://cd2688-object-detection-tf2/val/'} 

# Insert path of a folder in your personal S3 bucket to store tensorboard logs.
tensorboard_s3_prefix = 's3://urban-object-detection-project/logs/'

## Container

To train the model, you will first need to build a [docker](https://www.docker.com/) container with all the dependencies required by the TF Object Detection API. The code below does the following:
* clone the Tensorflow models repository
* get the exporter and training scripts from the repository
* build the docker image and push it 
* print the container name

In [5]:
%%bash

# clone the repo and get the scripts
git clone https://github.com/tensorflow/models.git docker/models

# get model_main and exporter_main files from TF2 Object Detection GitHub repository
cp docker/models/research/object_detection/exporter_main_v2.py source_dir 
cp docker/models/research/object_detection/model_main_tf2.py source_dir

fatal: destination path 'docker/models' already exists and is not an empty directory.


In [6]:
# build and push the docker image. This code can be commented out after being run once.
# This will take around 10 mins.
image_name = 'tf2-object-detection'
!sh ./docker/build_and_push.sh $image_name

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store

Login Succeeded
Building image with name tf2-object-detection
[+] Building 0.0s (0/1)                                          docker:default
[+] Building 0.1s (2/2)                                          docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
[+] Building 0.3s (5/15)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-

[+] Building 1.2s (5/15)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-agen  1.1s
 => => # Get:11 http://archive.ubuntu.com/ubuntu focal-backports InRelease [128
 => => #  kB]                                                                  
 => => # Get:12 http://archive.ubuntu.co

[+] Building 1.7s (5/15)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-agen  1.6s
 => => # Get:20 http://archive.ubuntu.com/ubuntu focal-updates/multiverse amd64
 => => #  Packages [34.6 kB]                                                   
 => => # Get:21 http://archive.ubuntu.co

[+] Building 2.6s (5/15)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-agen  2.5s
 => => # Get:22 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 Packa
 => => # ges [4639 kB]                                                         
 => => # Get:23 http://archive.ubuntu.co

[+] Building 3.4s (5/15)                                         docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-agen  3.3s
 => => # Get:23 http://archive.ubuntu.com/ubuntu focal-backports/main amd64 Pac
 => => # kages [55.2 kB]                                                       
 => => # Get:24 http://archive.ubuntu.co

...

[+] Building 103.5s (15/16)                                      docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-age  25.0s
 => [ 3/11] COPY models/research/object_detection /home/tensorflow/models  0.4s
 => [ 4/11] RUN (cd /home/tensorflow/models/research/ && protoc object_de  0.3s
 => [ 5/11] WORKDIR /home/tensorflow/mod

[+] Building 104.2s (15/16)                                      docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-age  25.0s
 => [ 3/11] COPY models/research/object_detection /home/tensorflow/models  0.4s
 => [ 4/11] RUN (cd /home/tensorflow/models/research/ && protoc object_de  0.3s
 => [ 5/11] WORKDIR /home/tensorflow/mod

[+] Building 105.0s (15/16)                                      docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-age  25.0s
 => [ 3/11] COPY models/research/object_detection /home/tensorflow/models  0.4s
 => [ 4/11] RUN (cd /home/tensorflow/models/research/ && protoc object_de  0.3s
 => [ 5/11] WORKDIR /home/tensorflow/mod

[+] Building 105.6s (15/16)                                      docker:default
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.01kB                                     0.0s
 => [internal] load metadata for docker.io/tensorflow/tensorflow:2.13.0-g  0.1s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 2B                                            0.0s
 => CACHED [ 1/11] FROM docker.io/tensorflow/tensorflow:2.13.0-gpu@sha256  0.0s
 => [internal] load build context                                          0.0s
 => => transferring context: 68.45kB                                       0.0s
 => [ 2/11] RUN apt-get update && apt-get install -y     git     gpg-age  25.0s
 => [ 3/11] COPY models/research/object_detection /home/tensorflow/models  0.4s
 => [ 4/11] RUN (cd /home/tensorflow/models/research/ && protoc object_de  0.3s
 => [ 5/11] WORKDIR /home/tensorflow/mod

f238b7a: Pushed   1.486GB/1.462GBPushing  87.45MB/335.6MBPushing  469.2MB/1.462GBPushing  1.277GB/1.462GB20241223172620: digest: sha256:7546cc07dbc4bbc73afdb451c620bdec0fd94504c6200129e2a607dc4e1b4e24 size: 6187
Saving ECR image URI into ecr_image_fullname.txt


To verify that the image was correctly pushed to the [Elastic Container Registry](https://aws.amazon.com/ecr/), you can look at it in the AWS webapp. For example, below you can see that three different images have been pushed to ECR. You should only see one, called `tf2-object-detection`.
![ECR Example](../data/example_ecr.png)


In [7]:
# display the container name
with open (os.path.join('docker', 'ecr_image_fullname.txt'), 'r') as f:
    container = f.readlines()[0][:-1]

print(container)

796973520885.dkr.ecr.us-east-1.amazonaws.com/tf2-object-detection:20241223172620


## Pre-trained model from model zoo

As often, we are not training from scratch and we will be using a pretrained model from the TF Object Detection model zoo. You can find pretrained checkpoints [here](https://github.com/tensorflow/models/blob/master/research/object_detection/g3doc/tf2_detection_zoo.md). Because your time is limited for this project, we recommend to only experiment with the following models:
* SSD MobileNet V2 FPNLite 640x640	
* SSD ResNet50 V1 FPN 640x640 (RetinaNet50)	
* Faster R-CNN ResNet50 V1 640x640	
* EfficientDet D1 640x640	
* Faster R-CNN ResNet152 V1 640x640	

In the code below, the EfficientDet D1 model is downloaded and extracted. This code should be adjusted if you were to experiment with other architectures.

In [8]:
%%bash
mkdir /tmp/checkpoint
mkdir source_dir/checkpoint
wget -O /tmp/efficientdet.tar.gz http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
tar -zxvf /tmp/efficientdet.tar.gz --strip-components 2 --directory source_dir/checkpoint efficientdet_d1_coco17_tpu-32/checkpoint

mkdir: cannot create directory ‘/tmp/checkpoint’: File exists
mkdir: cannot create directory ‘source_dir/checkpoint’: File exists
--2024-12-23 17:29:31--  http://download.tensorflow.org/models/object_detection/tf2/20200711/efficientdet_d1_coco17_tpu-32.tar.gz
Resolving download.tensorflow.org (download.tensorflow.org)... 142.250.31.207, 142.251.111.207, 142.251.163.207, ...
Connecting to download.tensorflow.org (download.tensorflow.org)|142.250.31.207|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 51839363 (49M) [application/x-tar]
Saving to: ‘/tmp/efficientdet.tar.gz’

     0K .......... .......... .......... .......... ..........  0% 15.6M 3s
    50K .......... .......... .......... .......... ..........  0% 29.8M 2s
   100K .......... .......... .......... .......... ..........  0% 30.4M 2s
   150K .......... .......... .......... .......... ..........  0% 31.1M 2s
   200K .......... .......... .......... .......... ..........  0% 31.2M 2s
   250K .........

  5000K .......... .......... .......... .......... ..........  9%  187M 1s
  5050K .......... .......... .......... .......... .......... 10%  166M 1s
  5100K .......... .......... .......... .......... .......... 10%  121M 1s
  5150K .......... .......... .......... .......... .......... 10%  127M 1s
  5200K .......... .......... .......... .......... .......... 10%  136M 1s
  5250K .......... .......... .......... .......... .......... 10%  163M 1s
  5300K .......... .......... .......... .......... .......... 10%  120M 1s
  5350K .......... .......... .......... .......... .......... 10%  210M 1s
  5400K .......... .......... .......... .......... .......... 10%  159M 1s
  5450K .......... .......... .......... .......... .......... 10%  110M 1s
  5500K .......... .......... .......... .......... .......... 10%  234M 1s
  5550K .......... .......... .......... .......... .......... 11%  147M 1s
  5600K .......... .......... .......... .......... .......... 11%  140M 1s
  5650K ....

 10400K .......... .......... .......... .......... .......... 20%  225M 0s
 10450K .......... .......... .......... .......... .......... 20%  145M 0s
 10500K .......... .......... .......... .......... .......... 20%  159M 0s
 10550K .......... .......... .......... .......... .......... 20%  198M 0s
 10600K .......... .......... .......... .......... .......... 21%  148M 0s
 10650K .......... .......... .......... .......... .......... 21%  163M 0s
 10700K .......... .......... .......... .......... .......... 21%  230M 0s
 10750K .......... .......... .......... .......... .......... 21%  157M 0s
 10800K .......... .......... .......... .......... .......... 21%  144M 0s
 10850K .......... .......... .......... .......... .......... 21%  224M 0s
 10900K .......... .......... .......... .......... .......... 21%  150M 0s
 10950K .......... .......... .......... .......... .......... 21%  198M 0s
 11000K .......... .......... .......... .......... .......... 21%  136M 0s
 11050K ....

 15800K .......... .......... .......... .......... .......... 31%  178M 0s
 15850K .......... .......... .......... .......... .......... 31%  127M 0s
 15900K .......... .......... .......... .......... .......... 31%  229M 0s
 15950K .......... .......... .......... .......... .......... 31%  157M 0s
 16000K .......... .......... .......... .......... .......... 31%  320M 0s
 16050K .......... .......... .......... .......... .......... 31%  139M 0s
 16100K .......... .......... .......... .......... .......... 31%  173M 0s
 16150K .......... .......... .......... .......... .......... 32%  144M 0s
 16200K .......... .......... .......... .......... .......... 32%  221M 0s
 16250K .......... .......... .......... .......... .......... 32%  184M 0s
 16300K .......... .......... .......... .......... .......... 32%  213M 0s
 16350K .......... .......... .......... .......... .......... 32%  149M 0s
 16400K .......... .......... .......... .......... .......... 32%  159M 0s
 16450K ....

 21200K .......... .......... .......... .......... .......... 41%  207M 0s
 21250K .......... .......... .......... .......... .......... 42%  231M 0s
 21300K .......... .......... .......... .......... .......... 42%  131M 0s
 21350K .......... .......... .......... .......... .......... 42%  164M 0s
 21400K .......... .......... .......... .......... .......... 42%  273M 0s
 21450K .......... .......... .......... .......... .......... 42%  168M 0s
 21500K .......... .......... .......... .......... .......... 42%  299M 0s
 21550K .......... .......... .......... .......... .......... 42%  179M 0s
 21600K .......... .......... .......... .......... .......... 42%  126M 0s
 21650K .......... .......... .......... .......... .......... 42%  274M 0s
 21700K .......... .......... .......... .......... .......... 42%  127M 0s
 21750K .......... .......... .......... .......... .......... 43%  206M 0s
 21800K .......... .......... .......... .......... .......... 43%  215M 0s
 21850K ....

...

 32000K .......... .......... .......... .......... .......... 63%  321M 0s
 32050K .......... .......... .......... .......... .......... 63%  197M 0s
 32100K .......... .......... .......... .......... .......... 63%  264M 0s
 32150K .......... .......... .......... .......... .......... 63%  182M 0s
 32200K .......... .......... .......... .......... .......... 63%  228M 0s
 32250K .......... .......... .......... .......... .......... 63%  309M 0s
 32300K .......... .......... .......... .......... .......... 63%  235M 0s
 32350K .......... .......... .......... .......... .......... 64%  228M 0s
 32400K .......... .......... .......... .......... .......... 64%  225M 0s
 32450K .......... .......... .......... .......... .......... 64%  233M 0s
 32500K .......... .......... .......... .......... .......... 64%  313M 0s
 32550K .......... .......... .......... .......... .......... 64%  214M 0s
 32600K .......... .......... .......... .......... .......... 64%  181M 0s
 32650K ....

 37400K .......... .......... .......... .......... .......... 73%  189M 0s
 37450K .......... .......... .......... .......... .......... 74%  231M 0s
 37500K .......... .......... .......... .......... .......... 74%  316M 0s
 37550K .......... .......... .......... .......... .......... 74%  210M 0s
 37600K .......... .......... .......... .......... .......... 74%  270M 0s
 37650K .......... .......... .......... .......... .......... 74%  317M 0s
 37700K .......... .......... .......... .......... .......... 74%  145M 0s
 37750K .......... .......... .......... .......... .......... 74%  215M 0s
 37800K .......... .......... .......... .......... .......... 74%  306M 0s
 37850K .......... .......... .......... .......... .......... 74%  265M 0s
 37900K .......... .......... .......... .......... .......... 74%  153M 0s
 37950K .......... .......... .......... .......... .......... 75%  208M 0s
 38000K .......... .......... .......... .......... .......... 75%  239M 0s
 38050K ....

 42800K .......... .......... .......... .......... .......... 84%  229M 0s
 42850K .......... .......... .......... .......... .......... 84%  311M 0s
 42900K .......... .......... .......... .......... .......... 84%  264M 0s
 42950K .......... .......... .......... .......... .......... 84%  204M 0s
 43000K .......... .......... .......... .......... .......... 85%  190M 0s
 43050K .......... .......... .......... .......... .......... 85%  226M 0s
 43100K .......... .......... .......... .......... .......... 85%  226M 0s
 43150K .......... .......... .......... .......... .......... 85%  263M 0s
 43200K .......... .......... .......... .......... .......... 85%  205M 0s
 43250K .......... .......... .......... .......... .......... 85%  228M 0s
 43300K .......... .......... .......... .......... .......... 85%  271M 0s
 43350K .......... .......... .......... .......... .......... 85%  164M 0s
 43400K .......... .......... .......... .......... .......... 85%  220M 0s
 43450K ....

 48200K .......... .......... .......... .......... .......... 95%  184M 0s
 48250K .......... .......... .......... .......... .......... 95%  222M 0s
 48300K .......... .......... .......... .......... .......... 95%  312M 0s
 48350K .......... .......... .......... .......... .......... 95%  203M 0s
 48400K .......... .......... .......... .......... .......... 95%  271M 0s
 48450K .......... .......... .......... .......... .......... 95%  294M 0s
 48500K .......... .......... .......... .......... .......... 95%  314M 0s
 48550K .......... .......... .......... .......... .......... 96%  265M 0s
 48600K .......... .......... .......... .......... .......... 96%  316M 0s
 48650K .......... .......... .......... .......... .......... 96%  314M 0s
 48700K .......... .......... .......... .......... .......... 96%  314M 0s
 48750K .......... .......... .......... .......... .......... 96%  269M 0s
 48800K .......... .......... .......... .......... .......... 96%  310M 0s
 48850K ....

efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-0.data-00000-of-00001
efficientdet_d1_coco17_tpu-32/checkpoint/checkpoint
efficientdet_d1_coco17_tpu-32/checkpoint/ckpt-0.index


## Edit pipeline.config file

The [`pipeline.config`](source_dir/pipeline.config) in the `source_dir` folder should be updated when you experiment with different models. The different config files are available [here](https://github.com/tensorflow/models/tree/master/research/object_detection/configs/tf2).

>Note: The provided `pipeline.config` file works well with the `EfficientDet` model. You would need to modify it when working with other models.

## Launch Training Job

Now that we have a dataset, a docker image and some pretrained model weights, we can launch the training job. To do so, we create a [Sagemaker Framework](https://sagemaker.readthedocs.io/en/stable/frameworks/index.html), where we indicate the container name, name of the config file, number of training steps etc.

The `run_training.sh` script does the following:
* train the model for `num_train_steps` 
* evaluate over the val dataset
* export the model

Different metrics will be displayed during the evaluation phase, including the mean average precision. These metrics can be used to quantify your model performances and compare over the different iterations.

You can also monitor the training progress by navigating to **Training -> Training Jobs** from the Amazon Sagemaker dashboard in the Web UI.

In [9]:
tensorboard_output_config = sagemaker.debugger.TensorBoardOutputConfig(
    s3_output_path=tensorboard_s3_prefix,
    container_local_output_path='/opt/training/'
)

estimator = CustomFramework(
    role=role,
    image_uri=container,
    entry_point='run_training.sh',
    source_dir='source_dir/',
    hyperparameters={
        "model_dir": "/opt/training",        
        "pipeline_config_path": "efficientdet_d1_aug.config",
        "num_train_steps": "2000",    
        "sample_1_of_n_eval_examples": "1"
    },
    instance_count=1,
    instance_type='ml.g5.xlarge',
    tensorboard_output_config=tensorboard_output_config,
    disable_profiler=True,
    base_job_name='tf2-object-detection'
)

estimator.fit(inputs)

[12/23/24 17:29:48] INFO     Found credentials from IAM Role:                                   ]8;id=10320;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=679276;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=773677;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=172067;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/botocore/credentials.py#1075\1075]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[12/23/24 17:29:49] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=448896;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=899094;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#90\90]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[12/23/24 17:29:50] INFO     Creating training-job with name:                                       ]8;id=290462;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=250606;file:///home/ec2-user/anaconda3/envs/tensorflow2_p310/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\
                             tf2-object-detection-2024-12-23-17-29-49-054                                          

2024-12-23 17:29:52 Starting - Starting the training job...
2024-12-23 17:30:07 Starting - Preparing the instances for training...
2024-12-23 17:30:47 Downloading - Downloading the training image............
2024-12-23 17:32:28 Training - Training image download completed. Training in progress.2024-12-23 17:32:39,730 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-12-23 17:32:39,766 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-12-23 17:32:39,802 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2024-12-23 17:32:39,817 sagemaker-training-toolkit INFO     Invoking user script
Training Env:
{
    "additional_framework_parameters": {},
    "channel_input_dirs": {
        "train": "/opt/ml/input/data/train",
        "val": "/opt/ml/input/data/val"
    },
    "current_host": "algo-1",
    "current_instance_group": "homogeneousCluster",
    "current_instance_gr

Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
W1223 17:32:56.034645 140495867197248 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sparse_to_dense (from tensorflow.python.ops.sparse_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Create a `tf.sparse.SparseTensor` and use `tf.sparse.to_dense` instead.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
W1223 17:32:59.055344 140495867197248 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: sample_distorted_bounding_box (from tensorflow.python.ops.image_ops_impl) is deprecated and will be removed in a future version.
Instructions for updating:
`seed2` arg is deprecated.Use sample_distorted_bounding_box_v2 instead.
Instructions for updating:
Use `tf.cast` instead.
W1223 17:33:01.495456 140495

INFO:tensorflow:Step 200 per-step time 0.666s
I1223 17:36:54.143666 140495867197248 model_lib_v2.py:705] Step 200 per-step time 0.666s
INFO:tensorflow:{'Loss/classification_loss': 0.41811818,
 'Loss/localization_loss': 0.019253207,
 'Loss/regularization_loss': 0.028984344,
 'Loss/total_loss': 0.4663557,
 'learning_rate': 0.0005}
I1223 17:36:54.143903 140495867197248 model_lib_v2.py:708] {'Loss/classification_loss': 0.41811818,
 'Loss/localization_loss': 0.019253207,
 'Loss/regularization_loss': 0.028984344,
 'Loss/total_loss': 0.4663557,
 'learning_rate': 0.0005}
INFO:tensorflow:Step 300 per-step time 0.665s
I1223 17:38:00.661506 140495867197248 model_lib_v2.py:705] Step 300 per-step time 0.665s
INFO:tensorflow:{'Loss/classification_loss': 0.34984338,
 'Loss/localization_loss': 0.011339028,
 'Loss/regularization_loss': 0.029059386,
 'Loss/total_loss': 0.3902418,
 'learning_rate': 0.0005}
I1223 17:38:00.661745 140495867197248 model_lib_v2.py:708] {'Loss/classification_loss': 0.34984338,

INFO:tensorflow:Step 1600 per-step time 0.665s
I1223 17:52:28.827091 140495867197248 model_lib_v2.py:705] Step 1600 per-step time 0.665s
INFO:tensorflow:{'Loss/classification_loss': 0.24903643,
 'Loss/localization_loss': 0.010315531,
 'Loss/regularization_loss': 0.0295843,
 'Loss/total_loss': 0.28893626,
 'learning_rate': 0.0005}
I1223 17:52:28.827338 140495867197248 model_lib_v2.py:708] {'Loss/classification_loss': 0.24903643,
 'Loss/localization_loss': 0.010315531,
 'Loss/regularization_loss': 0.0295843,
 'Loss/total_loss': 0.28893626,
 'learning_rate': 0.0005}
INFO:tensorflow:Step 1700 per-step time 0.665s
I1223 17:53:35.328798 140495867197248 model_lib_v2.py:705] Step 1700 per-step time 0.665s
INFO:tensorflow:{'Loss/classification_loss': 0.20850903,
 'Loss/localization_loss': 0.010284274,
 'Loss/regularization_loss': 0.02961721,
 'Loss/total_loss': 0.24841051,
 'learning_rate': 0.0005}
I1223 17:53:35.329041 140495867197248 model_lib_v2.py:708] {'Loss/classification_loss': 0.2085090

/usr/local/lib/python3.8/dist-packages/keras/src/backend.py:452: UserWarning: `tf.keras.backend.set_learning_phase` is deprecated and will be removed after 2020-10-11. To update it, simply pass a True/False value to the `training` argument of the `__call__` method of your layer or model.
  warnings.warn(
I1223 17:57:23.329056 140422337210176 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
I1223 17:57:36.477351 140422337210176 api.py:460] feature_map_spatial_dims: [(80, 80), (40, 40), (20, 20), (10, 10), (5, 5)]
Instructions for updating:
Use `tf.cast` instead.
W1223 17:57:41.761771 140422337210176 deprecation.py:364] From /usr/local/lib/python3.8/dist-packages/tensorflow/python/util/dispatch.py:1176: to_int64 (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
INFO:tensorflow:Finished eval step 0
I1223 17:57:41.801239 140422337210176 model_lib_v2.py:966] Fin

I1223 18:02:28.945556 139682700916544 efficientnet_model.py:143] round_filter input=24 output=24
I1223 18:02:28.945688 139682700916544 efficientnet_model.py:143] round_filter input=40 output=40
I1223 18:02:29.178029 139682700916544 efficientnet_model.py:143] round_filter input=40 output=40
I1223 18:02:29.178161 139682700916544 efficientnet_model.py:143] round_filter input=80 output=80
I1223 18:02:29.478337 139682700916544 efficientnet_model.py:143] round_filter input=80 output=80
I1223 18:02:29.478466 139682700916544 efficientnet_model.py:143] round_filter input=112 output=112
I1223 18:02:29.784662 139682700916544 efficientnet_model.py:143] round_filter input=112 output=112
I1223 18:02:29.784796 139682700916544 efficientnet_model.py:143] round_filter input=192 output=192
I1223 18:02:30.156621 139682700916544 efficientnet_model.py:143] round_filter input=192 output=192
I1223 18:02:30.156752 139682700916544 efficientnet_model.py:143] round_filter input=320 output=320
I1223 18:02:30.31311

You should be able to see your model training in the AWS webapp as shown below:
![ECR Example](../data/example_trainings.png)


## Improve on the initial model

Most likely, this initial experiment did not yield optimal results. However, you can make multiple changes to the `pipeline.config` file to improve this model. One obvious change consists in improving the data augmentation strategy. The [`preprocessor.proto`](https://github.com/tensorflow/models/blob/master/research/object_detection/protos/preprocessor.proto) file contains the different data augmentation method available in the Tf Object Detection API. Justify your choices of augmentations in the write-up.

Keep in mind that the following are also available:
* experiment with the optimizer: type of optimizer, learning rate, scheduler etc
* experiment with the architecture. The Tf Object Detection API model zoo offers many architectures. Keep in mind that the pipeline.config file is unique for each architecture and you will have to edit it.
* visualize results on the test frames using the `2_deploy_model` notebook available in this repository.

In the cell below, write down all the different approaches you have experimented with, why you have chosen them and what you would have done if you had more time and resources. Justify your choices using the tensorboard visualizations (take screenshots and insert them in your write-up), the metrics on the evaluation set and the generated animation you have created with [this tool](../2_run_inference/2_deploy_model.ipynb).

## Visualize with Tensorboard

In [1]:
!pip install tensorboard
## Visualize the tensor log on the terminal using  the tensorboard on the command prompt
## tensor board --logdir="path to tensor log file"

We visualize the log of the tf records found in "logs/efficient_d1_aug/train/" for the training log and "logs/efficient_d1_aug/eval/" for the evaluation logs as seen in the output below
## Sample of Training Images 
![efficient_d1_training_images](../figures/efficientdet_d1_aug/train/efficientdet_d1_aug_images.png)

## Training log
![efficientdet_d1_training_log](../figures/efficientdet_d1_aug/train/efficientdet_d1_aug_train.png)



### Sample of evaluation images
![efficientdet_d1_aug_evaluation_images](../figures/efficientdet_d1_aug/eval/efficientdet_d1_aug_images.png)

### Evaluation Log
![efficientdet_d1_aug_training_log](../figures/efficientdet_d1_aug/eval/efficientdet_d1_aug_eval.png)


### Model Report

**Data Augmentation**
Data Augmentation done for this model as seen in `source_dir/efficientdet_d1_aug.config` is :
1. Random Horizontal Flip

2. Random scale and crop and pad to square

3. Random vertical flip

4. Random adjust brightness

5. Random adjust contrast

#### Training log Report  

**Training Losses at step=2000:**

1. Classification Loss : **0.3725**

2. Localization Loss : **0.01193**

3. Regularization Loss : **0.02985**

4. Total Loss : **0.4143**

**Learning rate** : **5 x 10 ^ -4**

**Steps per seconds** : **1.503**



### Evaluation Log Report


**Validation Losses at step = 2000 :**

1. Classification Loss : **0.3011**

2. Localization Loss : **0.01915**

3. Regularization Loss : **0.02985**

4. Total Loss : **0.3501**

**Detection_Boxes_Precision**

1. mAP = **0.1033**

2. mAP( large) = **0.3955**

3. mAP (medium) = **0.4**

4. mAP (small) = **0.03769**

5. mAP @ 0.5IoU = **0.2237** 

6. mAP @ 0.75IoU = **0.08631**
